In [1]:
from hypex.dataset.dataset import Dataset, ExperimentData
from hypex.dataset.roles import (
    InfoRole,
    FeatureRole,
    TreatmentRole,
    PreTargetRole,
    TargetRole,
)
from hypex.experiments.aa_test import AA_TEST

In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(),
        "signup_month": FeatureRole(),
        "treat": TreatmentRole(),
        "pre_spends": PreTargetRole(),
        "post_spends": TargetRole(),
        "age": FeatureRole(),
        "gender": FeatureRole(),
        "industry": FeatureRole(),
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
data.roles

{'user_id': Info,
 'signup_month': Feature,
 'treat': Treatment,
 'pre_spends': PreTarget,
 'post_spends': Target,
 'age': Feature,
 'gender': Feature,
 'industry': Feature}

In [4]:
test = AA_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [5]:
result.analysis_tables

{'GroupDifference╰╰post_spends[AASplitter]':                                    B
 post_spends control mean  452.718578
 post_spends difference     -1.108044
 post_spends difference %   -0.244753
 post_spends test mean     451.610533,
 'TTest╰╰post_spends[AASplitter]':                                group  statistic   p-value   pass
 TTest╰╰post_spends[AASplitter]     B   1.406066  0.159736  False,
 'KSTest╰╰post_spends[AASplitter]':                                 group  statistic   p-value   pass
 KSTest╰╰post_spends[AASplitter]     B     0.0212  0.211152  False,
 'OneAASplitAnalyzer╰╰0':    TTest p-value  TTest pass  KSTest p-value  KSTest pass  mean test score
 0       0.159736         0.0        0.211152          0.0         0.194013}

In [ ]:
ed.data.data.sample(frac=1, random_state=1)

AttributeError: 'DataFrame' object has no attribute 'data'

In [ ]:
control_std = control_group.std()
test_std = test_group.std()

test_proportion = len(test_group) / (len(test_group) + len(control_group))
control_proportion = 1 - test_proportion

d = ((norm.ppf(1 - significance / 2) + norm.ppf(power)) / mde) ** 2
s = test_std ** 2 / test_proportion + control_std ** 2 / control_proportion
return d * s